# Extract North East 

In [26]:
import rioxarray, xarray
import geopandas
from shapely.geometry import box
from geocube.api.core import make_geocube

Set up a (local) `dask` cluster:

In [2]:
from dask.distributed import Client, LocalCluster
client = Client(LocalCluster())
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 16,Total memory: 62.49 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:35147,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 62.49 GiB
Comm: tcp://127.0.0.1:37789,Total threads: 4
Dashboard: http://127.0.0.1:34943/status,Memory: 15.62 GiB
Nanny: tcp://127.0.0.1:39407,


## Mosaic

Connect to mosaic:

In [5]:
mosaic_path = '/home/jovyan/data/ghs_composite_s2/GHS-composite-S2.vrt'
mosaic = rioxarray.open_rasterio(mosaic_path, chunks=True)

Pick bounds:

In [13]:
minx, miny, maxx, maxy = bb = geopandas.GeoSeries(
    [box(        
        -1.845703125, 
        54.835500081451656,
        -1.33209228515625,
        55.06421406528486
    )],
    crs='EPSG:4326'
).to_crs(
    mosaic.rio.crs
).total_bounds

Select bands and extent:

In [16]:
ne = mosaic.sel(
    band=[1, 2, 3],
    x=slice(minx, maxx),
    y=slice(maxy, miny)
).compute()

In [18]:
ne.rio.to_raster('ne_mosaic.tif', tiled=True)

Signatures as float:

In [46]:
ss['signature_type_f'] = (
    ss['k8'] * 10
) + ss['level2']

## Signatures

- Read vector

In [62]:
ss = geopandas.read_file(
    '/home/jovyan/data/spatial_signatures/'
    'spatial_signatures_GB.gpkg'
).to_crs(mosaic.rio.crs)

Simplified version (which is incomplete):

In [67]:
sss = geopandas.read_parquet(
    '/home/jovyan/data/spatial_signatures/'
    'signatures_form_combined_levels_simplified.pq'
).to_crs(mosaic.rio.crs)

- Build dictionary

In [87]:
types = {i: j[1]['type'] for i, j in enumerate(
    ss[['code', 'type']].drop_duplicates().iterrows()
)}
types_r = {types[i]: i for i in types}
types

{0: 'Countryside agriculture',
 1: 'Accessible suburbia',
 2: 'Open sprawl',
 3: 'Wild countryside',
 4: 'Warehouse/Park land',
 5: 'Gridded residential quarters',
 6: 'Urban buffer',
 7: 'Disconnected suburbia',
 8: 'Dense residential neighbourhoods',
 9: 'Connected residential neighbourhoods',
 10: 'Dense urban neighbourhoods',
 11: 'Local urbanity',
 12: 'Concentrated urbanity',
 13: 'Regional urbanity',
 14: 'outlier',
 15: 'Metropolitan urbanity',
 16: 'Hyper concentrated urbanity'}

In [93]:
from json import dump
with open('classes.json', 'w') as fo:
    dump(types, fo)

- Add float column

In [88]:
ss['code_no'] = ss['code'].map(types_r)

- Rasterise polygons

In [89]:
%%time
signatures_raster = make_geocube(
    ss, measurements=["code_no"], like=ne
)

CPU times: user 1min 10s, sys: 2.27 s, total: 1min 12s
Wall time: 1min 11s


- Write out

In [90]:
signatures_raster['code_no'].rio.to_raster('signatures.tif', tiled=True)